In [1]:
#импорт библиотек
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации

from sklearn import linear_model #линейные моделиё
from sklearn import tree #деревья решений
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики
from sklearn import preprocessing #предобработка
from sklearn.model_selection import train_test_split #сплитование выборки

import optuna
from sklearn.model_selection import cross_val_score
import hyperopt
from hyperopt import hp, fmin, tpe, Trials
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

%matplotlib inline
plt.style.use('seaborn')

/var/folders/t6/jc5fqrkn0gv95ps21cwdcp340000gn/T/ipykernel_12799/3319580802.py:22: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [2]:

data = pd.read_csv('data/_train_sem09.csv')

data.head()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


Так как данные все обработанны переходим сразу к моделям

In [3]:
X = data.drop(['Activity'], axis=1)
y = data['Activity']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state = 1, test_size = 0.2)

Логистическая регрессия

In [5]:
#Создаем объект класса логистическая регрессия
log_reg = linear_model.LogisticRegression(max_iter = 1000)
#Обучаем модель, минимизируя logloss
log_reg.fit(X_train, y_train)
print("accuracy на тестовом наборе: {:.2f}".format(log_reg.score(X_test, y_test)))
y_test_pred = log_reg.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

accuracy на тестовом наборе: 0.76
f1_score на тестовом наборе: 0.78


###  **GridSearchCV**

In [6]:
from sklearn.model_selection import GridSearchCV
param_grid = [{'penalty': ['l2', 'none'] , # тип регуляризации
                'solver': ['lbfgs', 'sag', 'newton-cg'], # алгоритм оптимизации
                'C': [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]}, # уровень силы регурялизации
              
              {'penalty': ['l1', 'l2'] ,
                'solver': ['liblinear', 'saga'],
                'C': [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]}
]
grid_search = GridSearchCV(
    estimator=linear_model.LogisticRegression(
        random_state=42, #генератор случайных чисел
        max_iter=50 #количество итераций на сходимость
    ), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)  
%time grid_search.fit(X_train, y_train) 
print("accuracy на тестовом наборе: {:.2f}".format(grid_search.score(X_test, y_test)))
y_test_pred = grid_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search.best_params_))

/Users/sev/Library/Python/3.8/lib/python/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/sev/Library/Python/3.8/lib/python/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logis

CPU times: user 3.59 s, sys: 637 ms, total: 4.23 s
Wall time: 10min 32s
accuracy на тестовом наборе: 0.77
f1_score на тестовом наборе: 0.79
Наилучшие значения гиперпараметров: {'C': 0.05, 'penalty': 'l2', 'solver': 'lbfgs'}


###  **RandomizedSearchCV**

In [7]:
from sklearn.model_selection import RandomizedSearchCV

In [8]:

#np.linspace(start(от), stop(до), num=50(количество),dtype- тип данных)
param_distributions = [{'penalty': ['l2', 'none'] , # тип регуляризации
              'solver': ['lbfgs', 'sag', 'newton-cg'], # алгоритм оптимизации
               'C': [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]}, # уровень силы регурялизации
              
              {'penalty': ['l1', 'l2'] ,
              'solver': ['liblinear', 'saga'],
               'C': [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]}
]
            
random_search = RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42, max_iter=50), 
    param_distributions=param_distributions, 
    cv=5, 
    n_iter = 50, 
    n_jobs = -1
)  
%time random_search.fit(X_train, y_train) 
print("accuracy на тестовом наборе: {:.2f}".format(random_search.score(X_test, y_test)))
y_test_pred = random_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search.best_params_))

/Users/sev/Library/Python/3.8/lib/python/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/sev/Library/Python/3.8/lib/python/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/sev/Library/Python/3.8/lib/python/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/sev/Library/Python/3.8/lib/python/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/sev/Library/Python/3.8/lib/python/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/sev/Library/Python/3.8/lib/python/site-pack

CPU times: user 2.51 s, sys: 316 ms, total: 2.82 s
Wall time: 4min 6s
accuracy на тестовом наборе: 0.77
f1_score на тестовом наборе: 0.79
Наилучшие значения гиперпараметров: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 0.05}


Hyperopt

In [9]:
# зададим пространство поиска гиперпараметров

space = {'penalty': hp.choice('penalty', ['l1', 'l2']) ,
              'solver': hp.choice('solver', ['liblinear', 'saga']),
              'C': hp.uniform('C', 0.01, 1)
              }

In [10]:
def hyperopt(space, cv=5, X=X, y=y, random_state=42):
    """ Функция, обучающая модель LogisticRegression
    по переданным гиперпараметрам

    Args:
        space (dict): набор гиперпараметров
        cv (int, optional=5): Количество холдов кросс-валидации. Defaults to 5.
        X (DataFrame): DataFrame с признаками. Defaults to X_train.
        y (Series): Series с целевым признаком. Defaults to y_train.
        random_state (int): Рандомное число для воспроизводимости результата. Defaults to random_state.

    Returns:
        score(float): метрика F1
    """
    try:
        model=linear_model.LogisticRegression(
        penalty = space['penalty'],
        solver = space['solver'],
        C=np.abs(float(space['C'])),
        l1_ratio=float(space['l1_ratio']),
        random_state=random_state,
        max_iter=50        
    )        
    except KeyError:
          
        model=linear_model.LogisticRegression(
        penalty = space['penalty'],
        solver = space['solver'],
        C=np.abs(float(space['C'])),
        random_state=random_state,
        max_iter=50
           )
        
     
    # применим  cross validation 
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()
    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

In [11]:
%config Application.log_level='ERROR'
# начинаем подбор гиперпараметров
#%%time
random_state=42
trials = Trials() # используется для логирования результатов

best=fmin(hyperopt, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=50, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))

  2%|▏         | 1/50 [00:01<01:02,  1.27s/trial, best loss: -0.7790610933297455]

/Users/sev/Library/Python/3.8/lib/python/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


100%|██████████| 50/50 [04:12<00:00,  5.04s/trial, best loss: -0.7891514597312479]
Наилучшие значения гиперпараметров {'C': 0.21891904268225607, 'penalty': 0, 'solver': 1}


In [12]:
penalty=['l1','l2']
solver=['liblinear', 'saga']

In [13]:
# рассчитаем точность для тестовой выборки
model = linear_model.LogisticRegression(
    penalty = penalty[best['penalty']],
    solver = solver[best['solver']],
    C=best['C'],
    random_state=random_state,
    max_iter=50
)
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на обучающем наборе: 0.82
accuracy на тестовом наборе: 0.76
f1_score на тестовом наборе: 0.79


Optuna

In [26]:
import optuna

In [53]:
def optuna_rf(trial):
  # задаем пространства поиска гиперпараметров
  penalty =trial.suggest_categorical("penalty",['l1','l2'])
  solver =trial.suggest_categorical("solver",['liblinear', 'saga'])
  C= trial.suggest_float("C",0.1, 1)

  # создаем модель
  model = linear_model.LogisticRegression(penalty=penalty,
                                          solver=solver,
                                          C=C,
                                          random_state=42)
  # обучаем модель
  model.fit(X_train, y_train)
  score = metrics.f1_score(y_train, model.predict(X_train))

  return score

In [58]:
%%time
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="LogisticRegression", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=50)

[I 2023-02-18 15:04:57,346] A new study created in memory with name: LogisticRegression
[I 2023-02-18 15:04:57,994] Trial 0 finished with value: 0.8807339449541284 and parameters: {'penalty': 'l2', 'solver': 'liblinear', 'C': 0.7263454388292588}. Best is trial 0 with value: 0.8807339449541284.
[I 2023-02-18 15:04:58,374] Trial 1 finished with value: 0.8094951923076923 and parameters: {'penalty': 'l1', 'solver': 'liblinear', 'C': 0.14085121795729016}. Best is trial 0 with value: 0.8807339449541284.
[I 2023-02-18 15:04:58,866] Trial 2 finished with value: 0.8711355984083258 and parameters: {'penalty': 'l2', 'solver': 'liblinear', 'C': 0.422890674972877}. Best is trial 0 with value: 0.8807339449541284.
[I 2023-02-18 15:05:07,574] Trial 3 finished with value: 0.850546780072904 and parameters: {'penalty': 'l1', 'solver': 'saga', 'C': 0.7320798491735853}. Best is trial 0 with value: 0.8807339449541284.
[I 2023-02-18 15:05:15,415] Trial 4 finished with value: 0.8489820723184444 and parameters

CPU times: user 1min 40s, sys: 4.28 s, total: 1min 44s
Wall time: 1min 40s


In [59]:
# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study.best_params))
print("f1_score на обучающем наборе: {:.2f}".format(study.best_value))

Наилучшие значения гиперпараметров {'penalty': 'l2', 'solver': 'liblinear', 'C': 0.9994164323248411}
f1_score на обучающем наборе: 0.89


In [61]:
# рассчитаем точность для тестовой выборки
model = linear_model.LogisticRegression(**study.best_params,random_state=random_state )
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

accuracy на тестовом наборе: 0.76
f1_score на тестовом наборе: 0.78


Посмотрим второй набор парамметров для регрессии.

In [62]:
def optuna_rf2(trial):
  # задаем пространства поиска гиперпараметров
  penalty =trial.suggest_categorical("penalty",['none','l2'])
  solver =trial.suggest_categorical("solver",['lbfgs', 'sag', 'newton-cg'])
  C= trial.suggest_float("C",0.1, 1)

  # создаем модель
  model = linear_model.LogisticRegression(penalty=penalty,
                                          solver=solver,
                                          C=C,
                                          random_state=42)
  # обучаем модель
  model.fit(X_train, y_train)
  score = metrics.f1_score(y_train, model.predict(X_train))

  return score

In [63]:
%%time
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="LogisticRegression", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf2, n_trials=50)

[I 2023-02-18 15:14:49,624] A new study created in memory with name: LogisticRegression
[I 2023-02-18 15:14:54,186] Trial 0 finished with value: 0.8495145631067962 and parameters: {'penalty': 'l2', 'solver': 'sag', 'C': 0.10547906147422154}. Best is trial 0 with value: 0.8495145631067962.
[I 2023-02-18 15:14:54,664] Trial 1 finished with value: 0.8689275893675528 and parameters: {'penalty': 'l2', 'solver': 'lbfgs', 'C': 0.3466342137833449}. Best is trial 1 with value: 0.8689275893675528.
[I 2023-02-18 15:14:59,596] Trial 2 finished with value: 0.8754208754208753 and parameters: {'penalty': 'l2', 'solver': 'sag', 'C': 0.7169616314589594}. Best is trial 2 with value: 0.8754208754208753.
[I 2023-02-18 15:15:05,244] Trial 3 finished with value: 0.8856269113149847 and parameters: {'penalty': 'none', 'solver': 'sag', 'C': 0.7189706981362589}. Best is trial 3 with value: 0.8856269113149847.
[I 2023-02-18 15:15:10,743] Trial 4 finished with value: 0.8763769889840881 and parameters: {'penalty':

CPU times: user 22min 20s, sys: 1min 16s, total: 23min 36s
Wall time: 7min 23s


In [64]:
# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study.best_params))
print("f1_score на обучающем наборе: {:.2f}".format(study.best_value))

Наилучшие значения гиперпараметров {'penalty': 'none', 'solver': 'newton-cg', 'C': 0.5103751682496341}
f1_score на обучающем наборе: 1.00


In [65]:
# рассчитаем точность для тестовой выборки
model = linear_model.LogisticRegression(**study.best_params,random_state=random_state )
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

accuracy на тестовом наборе: 0.70
f1_score на тестовом наборе: 0.73


/Users/sev/Library/Python/3.8/lib/python/site-packages/scipy/optimize/_linesearch.py:305: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)


#### **Случайный лес**

Проделаем аналогичное для RandomForestClassifier().
Сначала посчитаем модель с параметрами по умолчанию и оценим метрику:

In [14]:
#Создаем объект класса случайный лес
rf = ensemble.RandomForestClassifier(random_state=42)

#Обучаем модель
rf.fit(X_train, y_train)
#Выводим значения метрики 
y_train_pred = rf.predict(X_train)
print('Train: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = rf.predict(X_test)
print('Test: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

Train: 1.00
Test: 0.81


GridSearchCV

In [20]:
param_grid = {'n_estimators': list(range(80, 200, 30)),
              'min_samples_leaf': [4,5,6,7],
              'max_depth': list(np.linspace(20, 40, 5, dtype=int))
              }
            
grid_search_forest = GridSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)  
%time grid_search_forest.fit(X_train, y_train) 
y_train_pred = grid_search_forest.predict(X_train)
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(grid_search_forest.score(X_test, y_test)))
y_test_pred = grid_search_forest.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_forest.best_params_))

CPU times: user 3.89 s, sys: 267 ms, total: 4.16 s
Wall time: 2min 33s
f1_score на обучающем наборе: 0.96
accuracy на тестовом наборе: 0.81
f1_score на тестовом наборе: 0.83
Наилучшие значения гиперпараметров: {'max_depth': 30, 'min_samples_leaf': 4, 'n_estimators': 170}


RandomizedSearchCV

In [21]:
param_distributions = {'n_estimators': list(range(80, 200, 30)),
              'min_samples_leaf': [4,5,6,7],
              'max_depth': list(np.linspace(20, 40, 10, dtype=int))
              }
            
random_search_forest = RandomizedSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_distributions=param_distributions, 
    cv=5,
    n_iter = 10, 
    n_jobs = -1
)  
%time random_search_forest.fit(X_train, y_train) 
y_train_pred = random_search_forest.predict(X_train)
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(random_search_forest.score(X_test, y_test)))
y_test_pred = random_search_forest.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search_forest.best_params_))

CPU times: user 3.28 s, sys: 186 ms, total: 3.46 s
Wall time: 1min 39s
f1_score на обучающем наборе: 0.96
accuracy на тестовом наборе: 0.81
f1_score на тестовом наборе: 0.83
Наилучшие значения гиперпараметров: {'n_estimators': 170, 'min_samples_leaf': 4, 'max_depth': 35}


Hyperopt

In [66]:
# зададим пространство поиска гиперпараметров
space={'n_estimators': hp.quniform('n_estimators', 100, 200, 1),
       'max_depth' : hp.quniform('max_depth', 3, 20, 1),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 2, 14, 1)
      }

In [67]:
# зафксируем random_state
random_state = 42
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'min_samples_leaf': int(params['min_samples_leaf'])
              }
  
    # используем эту комбинацию для построения модели
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)

    # обучаем модель
    model.fit(X, y)
    score = metrics.f1_score(y, model.predict(X))
    

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

In [68]:
# начинаем подбор гиперпараметров
#%%time

trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_rf, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=50, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))

100%|██████████| 50/50 [01:56<00:00,  2.34s/trial, best loss: -0.9914163090128756]
Наилучшие значения гиперпараметров {'max_depth': 20.0, 'min_samples_leaf': 2.0, 'n_estimators': 115.0}


In [69]:
# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(
    random_state=random_state, 
    n_estimators=int(best['n_estimators']),
    max_depth=int(best['max_depth']),
    min_samples_leaf=int(best['min_samples_leaf'])
)
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на обучающем наборе: 0.99
accuracy на тестовом наборе: 0.80
f1_score на тестовом наборе: 0.82


Optuna

In [70]:
def optuna_rf(trial):
  # задаем пространства поиска гиперпараметров
  n_estimators = trial.suggest_int('n_estimators', 100, 200, 1)
  max_depth = trial.suggest_int('max_depth', 1, 30, 1)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 20, 1)

  # создаем модель
  model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=random_state)
  # обучаем модель
  model.fit(X_train, y_train)
  score = metrics.f1_score(y_train, model.predict(X_train))

  return score
  
  

In [72]:
%%time
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=50)

[I 2023-02-18 15:42:31,477] A new study created in memory with name: RandomForestClassifier
[I 2023-02-18 15:42:32,857] Trial 0 finished with value: 0.8014773776546629 and parameters: {'n_estimators': 187, 'max_depth': 5, 'min_samples_leaf': 16}. Best is trial 0 with value: 0.8014773776546629.
[I 2023-02-18 15:42:35,570] Trial 1 finished with value: 0.91531641699786 and parameters: {'n_estimators': 199, 'max_depth': 29, 'min_samples_leaf': 8}. Best is trial 1 with value: 0.91531641699786.
[I 2023-02-18 15:42:37,751] Trial 2 finished with value: 0.871559633027523 and parameters: {'n_estimators': 147, 'max_depth': 25, 'min_samples_leaf': 14}. Best is trial 1 with value: 0.91531641699786.
[I 2023-02-18 15:42:38,997] Trial 3 finished with value: 0.818463444857497 and parameters: {'n_estimators': 117, 'max_depth': 6, 'min_samples_leaf': 14}. Best is trial 1 with value: 0.91531641699786.
[I 2023-02-18 15:42:41,046] Trial 4 finished with value: 0.9134821155609905 and parameters: {'n_estimator

CPU times: user 1min 48s, sys: 3.24 s, total: 1min 51s
Wall time: 2min


In [73]:
# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study.best_params))
print("f1_score на обучающем наборе: {:.2f}".format(study.best_value))

Наилучшие значения гиперпараметров {'n_estimators': 173, 'max_depth': 20, 'min_samples_leaf': 2}
f1_score на обучающем наборе: 0.99


In [75]:
# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(**study.best_params,random_state=random_state, )
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

accuracy на тестовом наборе: 0.80
f1_score на тестовом наборе: 0.82
